In [1]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
from multiprocessing import Pool

In [2]:
def scrape(url):
    
    # Lists to store the scraped data in
    makes = []
    models = []
    prices = []
    ratings = []
    dates = []
    types = []
    miles = []
    litres = []
    bhps = []
    transmissions = []
    fuels = []
    owners = []
    ulezs = []

    # Attempt to connect to the url
    try:
        response = get(url)
    except:
        print('oops')
    html_soup = BeautifulSoup(response.text, 'html.parser')

    # Get a list of individual cars and iterate through it
    car_containers = html_soup.find_all('li', class_ = 'search-page__result')
        
    for container in car_containers:
        try:
            container.find("div", {"class": "js-tooltip"}).find("div", {"class": "pi-indicator js-tooltip-trigger"}).text
            rating = container.find("div", {"class": "js-tooltip"}).find("div", {"class": "pi-indicator js-tooltip-trigger"}).text.strip()
        except:
            rating = ''
        ratings.append(rating)
        
        make = container.h2.text.strip().title().split(' ')[0]
        makes.append(make)
        
        model = container.h2.text.strip().title().split(' ')[1]
        models.append(model)

        price = container.find("div", {"class": "vehicle-price"}).text[1:]
        prices.append(price)

        specs = container.find("ul", {"class": "listing-key-specs"}).find_all("li", recursive=True)

        for spec in specs:
            
            if spec.text.split(' ')[0].isdigit() and len(spec.text.split(' ')[0]) == 4:
                date = spec.text.split(' ')[0]
                dates.append(date)

            if 'mile' in str(spec):
                mile = spec.text.split(' ')[0]
                miles.append(mile)

            if 'l' in str(spec).lower() and str(spec.text)[:-1].replace('.', '').isnumeric() and not spec.text.split(' ')[0].isdigit():
                litre = spec.text[:-1]
                litres.append(litre)

            if any(x in str(spec).lower() for x in ['automatic', 'manual']):
                transmission = spec.text
                transmissions.append(transmission)

            if any(x in str(spec).lower() for x in ['bhp', 'ps']):
                bhp = spec.text
                bhps.append(bhp)

            if any(x in str(spec).lower() for x in ['petrol', 'diesel']):
                fuel = spec.text
                fuels.append(fuel)
                
            if 'owner' in str(spec).lower():
                owner = spec.text
                owners.append(owner.split(' ')[0])
            
            if 'ulez' in str(spec).lower():
                ulez = spec.text
                ulezs.append(ulez)
            
        
            typelist = ['hatchback', 'saloon', 'convertible', 'coupe', 'suv', 'mpv', 'estate', 'limousine',
                        'pickup']
            if any(x in str(spec).lower() for x in typelist):
                typ = spec.text
                types.append(typ)
            
        
        attributes = [makes, models, ratings, dates, types, miles, litres, bhps, transmissions,
                      fuels, owners, ulezs]

        # Filling in empty spaces
        for attribute in attributes:
            if len(attribute) < len(prices):
                attribute.append('')
                
    # Creating a dataframe from the lists
    df = ({'Make': makes,
           'Model': models,
           'Price': prices,
            'Rating': ratings,
            'Year': dates,
            'Type': types,
            'Miles': miles,
            'Litres': litres,
            'BHP': bhps,
            'Transmission': transmissions,
            'Fuel': fuels,
            'Owners': owners,
            'ULEZ': ulezs
    })
    
    return df

In [3]:
# List of urls
path = 'https://www.autotrader.co.uk/car-search?advertClassification=standard&postcode=RH104JJ&make=FORD&price-from=500&price-to=1500&model=FOCUS&onesearchad=Used&onesearchad=Nearly%20New&onesearchad=New&advertising-location=at_cars&is-quick-search=TRUE&page='
urls = []
for i in range(1,58):
    url = path + str(i)
    urls.append(url)  

In [4]:
p = Pool(19)
df = p.map(scrape, urls)
p.terminate()
p.join()

In [5]:
len(df[55]['Price'])

11

In [6]:
makes = []
models = []
prices = []
ratings = []
dates = []
types = []
miles = []
litres = []
bhps = []
transmissions = []
fuels = []
owners = []
ulezs = []

for i in range(len(df)):
    for j in range(len(df[i])):
        try:
            makes.append(df[i]['Make'][j])
            models.append(df[i]['Model'][j])
            prices.append(df[i]['Price'][j])
            ratings.append(df[i]['Rating'][j])
            dates.append(df[i]['Year'][j])
            types.append(df[i]['Type'][j])
            miles.append(df[i]['Miles'][j])
            litres.append(df[i]['Litres'][j])
            bhps.append(df[i]['BHP'][j])
            transmissions.append(df[i]['Transmission'][j])
            fuels.append(df[i]['Fuel'][j])
            owners.append(df[i]['Owners'][j])
            ulezs.append(df[i]['ULEZ'][j])
        except:
            makes.append('')
            models.append('')
            prices.append('')
            ratings.append('')
            dates.append('')
            types.append('')
            miles.append('')
            litres.append('')
            bhps.append('')
            transmissions.append('')
            fuels.append('')
            owners.append('')
            ulezs.append('')


# Creating a dataframe from the lists
df = ({'Make': makes,
       'Model': models,
       'Price': prices,
        'Rating': ratings,
        'Year': dates,
        'Type': types,
        'Miles': miles,
        'Litres': litres,
        'BHP': bhps,
        'Transmission': transmissions,
        'Fuel': fuels,
        'Owners': owners,
        'ULEZ': ulezs
})
df = pd.DataFrame(df)

In [7]:
df

,Make,Model,Price,Rating,Year,Type,Miles,Litres,BHP,Transmission,Fuel,Owners,ULEZ
0,Ford,Focus,550,,2000,Hatchback,458,1.6,99BHP,Automatic,Petrol,6,
1,Ford,Focus,550,,2007,Hatchback,"105,030",1.6,99BHP,Manual,Petrol,9,ULEZ
2,Ford,Focus,"1,500",,2001,Hatchback,"70,500",1.6,99BHP,Automatic,Petrol,3,
3,Ford,Focus,800,,2003,Hatchback,"78,453",1.6,,Automatic,Petrol,3,
4,Ford,Focus,750,,2008,Hatchback,"161,000",1.6,108BHP,Manual,Diesel,4,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
736,Ford,Focus,"1,295",,2007,Hatchback,"94,000",1.6,99BHP,Manual,Petrol,,ULEZ
737,Ford,Focus,950,,2005,Hatchback,"118,000",1.6,99BHP,Manual,Petrol,7,ULEZ
738,Ford,Focus,"12,800",Good price,2017,Hatchback,"12,596",1.0,125PS,Automatic,Petrol,1,ULEZ
739,,,,,,,,,,,,,


In [8]:
df.replace('', float('NaN'), inplace=True)
df.dropna(inplace=True, how = 'all')

In [9]:
df

,Make,Model,Price,Rating,Year,Type,Miles,Litres,BHP,Transmission,Fuel,Owners,ULEZ
0,Ford,Focus,550,NaN,2000,Hatchback,458,1.6,99BHP,Automatic,Petrol,6,NaN
1,Ford,Focus,550,NaN,2007,Hatchback,"105,030",1.6,99BHP,Manual,Petrol,9,ULEZ
2,Ford,Focus,"1,500",NaN,2001,Hatchback,"70,500",1.6,99BHP,Automatic,Petrol,3,NaN
3,Ford,Focus,800,NaN,2003,Hatchback,"78,453",1.6,NaN,Automatic,Petrol,3,NaN
4,Ford,Focus,750,NaN,2008,Hatchback,"161,000",1.6,108BHP,Manual,Diesel,4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
734,Ford,Focus,"1,100",NaN,2007,Hatchback,"108,000",1.6,99BHP,Manual,Petrol,NaN,ULEZ
735,Ford,Focus,750,NaN,2007,Hatchback,"120,000",1.6,99BHP,Manual,Petrol,7,ULEZ
736,Ford,Focus,"1,295",NaN,2007,Hatchback,"94,000",1.6,99BHP,Manual,Petrol,NaN,ULEZ
737,Ford,Focus,950,NaN,2005,Hatchback,"118,000",1.6,99BHP,Manual,Petrol,7,ULEZ


In [10]:
df.to_csv('Ford_Focus_data')